In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sm
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
train_df = pd.read_csv("hw4/train.csv")
test_df = pd.read_csv("hw4/test.csv")

train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
1,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,0
2,1240,2,"Giles, Mr. Ralph",male,24.0,0,0,248726,13.5000,NaN,S,0
3,221,3,"Sunderland, Mr. Victor Francis",male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S,1
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S,0


### Работа с признаками

Как видно выше, поле Name содержит в себе не только имя, но и титул, поэтому было бы неплохо эту информацию выделить
в отдельную колонку.


In [3]:
train_df["Title"] = train_df.Name.apply(lambda x: x.split(" ")[1].replace(".", ""))
test_df["Title"] = test_df.Name.apply(lambda x: x.split(" ")[1].replace(".", ""))

Посчитаем сколько у нас титулов имеется

In [4]:
title_stat = train_df.groupby("Title").Name.agg(["count"])
title_stat

,count
Title,
"Billiard,",2
"Carlo,",2
Col,3
Dr,5
"Gordon,",1
"Impe,",1
"Khalil,",1
Major,1
Master,38


Как видим, достаточно много титулов являются редкими, поэтому их нужно объединить в один. Сделаем это

In [5]:
title_stat["is_rare"] = title_stat["count"] < 10
titles = title_stat[~title_stat.is_rare].index.values

train_df.Title = train_df.Title.apply(lambda x: x if x in titles else "rare_title")
test_df.Title = test_df.Title.apply(lambda x: x if x in titles else "rare_title")

title_stat = train_df.groupby("Title").Name.agg(["count"])
title_stat

,count
Title,
Master,38
Miss,163
Mr,487
Mrs,123
rare_title,39


In [6]:
test_df.groupby("Title").Name.agg(["count"])

,count
Title,
Master,21
Miss,93
Mr,249
Mrs,68
rare_title,28


Если вы смотрели фильм, то наверняка помните, что в первую очередь на лодки сажали
женщин и детей, а также родителей с детьми. Было бы неплохо данную информацию отразить в признаках.

In [7]:
train_df.Cabin.astype("str")

0                  B42
1                  nan
2                  nan
3                  nan
4                  nan
5                  nan
6                  nan
7                  E40
8                  nan
9                  nan
10                 nan
11                 E25
12                 nan
13                 nan
14                 nan
15                 E34
16                 nan
17                 nan
18                 nan
19                 nan
20                 nan
21                 nan
22                 nan
23                 nan
24     B57 B59 B63 B66
25             B58 B60
26                 nan
27                 nan
28                 nan
29                 nan
            ...       
820               B101
821                nan
822                nan
823            C22 C26
824                nan
825                nan
826                nan
827                nan
828                nan
829                nan
830    B57 B59 B63 B66
831                nan
832        

In [8]:
train_df["Children"] = train_df.Age.apply(lambda x: 1 if x < 18 else 0)
train_df["Mother"] = train_df.apply(lambda x: 1 if x.Sex == "female" and x.Age > 18 and x.Parch > 0 else 0, axis=1)
train_df["Family_size"] = train_df.SibSp + train_df.Parch + 1

test_df["Children"] = test_df.Age.apply(lambda x: 1 if x < 18 else 0)
test_df["Mother"] = test_df.apply(lambda x: 1 if x.Sex == "female" and x.Age > 18 and x.Parch > 0 else 0, axis=1)
test_df["Family_size"] = test_df.SibSp + test_df.Parch + 1

Теперь вспомним, что у нас есть также номер кабинки. Первая буква в номере - что-то очень похожее на блок, а блоки расположены в разных частях корабля. Данный признак потенциально может быть полезен для нас

In [9]:
train_df["Deck"] = train_df.Cabin.astype("str").apply(lambda x: x[0] if x != "nan" else "no_deck")
test_df["Deck"] = test_df.Cabin.astype("str").apply(lambda x: x[0] if x != "nan" else "no_deck")

Далее удалим ненужные исходные колонки, которыми мы уже воспользовались при работе с признаками.
Помимо этого нужно помнить, что в наших данных есть пропуски, которые необходимо устранить, т.к.
что LogReg, что KNN с ними работать не умеют, в отличие от деревьев.

Воспользуемся самым простым - заполним все пропуски нулями. Есть и другие способы, такие как
среднее значение, медианное значение, среднее по какой-то группе, обучение и использование классификатора
для заполнения пропусков. Они потенциально дают больший скор, но пока что воспользуемся самым простым.

In [10]:
def preprocessing(df):
    unnecessary_cols = ["Name", "Ticket", "SibSp", "Parch", "Cabin"]

    df = df.drop(unnecessary_cols, axis=1)
    df.Age = df.Age.fillna(0).astype("int8")
    df.Fare = df.Fare.fillna(0)
    df.Embarked = df.Embarked.fillna("no_info")
    
    return df

train_df = preprocessing(train_df)
test_df = preprocessing(test_df)

Теперь вспомним, что в наших датафреймах содержатся категориальные признаки в виде строк, с которыми
линейные модели и KNN работать не умеют. Нам их необходимо преобразовать от строк к численным значениям.
Можно это сделать руками, а альтернатива - воспользоваться **LabelEncoder** из *sklearn*, который данное преобразование
из коробки сделает за нас.

In [11]:
encoder = LabelEncoder()
encoding_cols = ["Sex", "Embarked", "Title", "Deck"]

full_df = pd.concat([train_df, test_df], axis=0)

for col in encoding_cols:
    full_df[col] = encoder.fit_transform(full_df[col])
    
train_df = full_df[full_df.Survived.notnull()]
train_df.Survived = train_df.Survived.astype("int8")

test_df = full_df[full_df.Survived.isnull()]
test_df.drop("Survived", axis=1, inplace=True)

train_df.head()

,Age,Children,Deck,Embarked,Family_size,Fare,Mother,PassengerId,Pclass,Sex,Survived,Title
0,19,0,1,2,1,30.0000,0,888,1,0,1,1
1,0,0,8,2,1,7.8792,0,1249,3,1,0,2
2,24,0,8,2,1,13.5000,0,1240,2,1,0,2
3,16,1,8,2,1,8.0500,0,221,3,1,1,2
4,60,0,8,2,2,26.0000,0,1105,2,0,0,3


In [12]:
test_df.head()

,Age,Children,Deck,Embarked,Family_size,Fare,Mother,PassengerId,Pclass,Sex,Title
0,20,0,8,2,2,26.00,0,1167,2,0,1
1,33,0,8,2,1,26.55,0,1215,1,1,2
2,38,0,8,2,1,0.00,0,823,1,1,4
3,0,0,8,2,11,69.55,0,864,3,0,1
4,4,1,6,2,3,16.70,0,11,3,0,1


In [13]:
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

def run_cv(X, y, model, verbose=False):
    folds = StratifiedKFold(5)
    scores = []

    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train, y_train = (X[train_idx], y[train_idx])
        X_val, y_val = (X[val_idx], y[val_idx])
    
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
    
        score = sm.accuracy_score(y_val, y_pred)
        scores.append(score)
    
        if verbose:
            print(f"Fold {i}: Accuracy - {score}, {knn_score}")
        
    return scores

scaler = StandardScaler()
model = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=20)

feature_cols = [col for col in train_df.columns if col not in {"PassengerId", "Survived"}]
target_col = "Survived"

X = train_df[feature_cols].values
X = scaler.fit_transform(X)
y = train_df[target_col]

Подберем на валидации оптимальные параметры для KNN и найдем наилучший средний скор (понадобится для сравнения с LogReg)

In [14]:
k_values = np.arange(5, 100, 1)

best_score = 0
best_score_std = 0
best_k = 0

for k in tqdm(k_values):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn_scores = run_cv(X, y, knn)
    avg_score = np.mean(knn_scores)
    
    if avg_score > best_score:
        best_score = avg_score
        best_score_std = np.std(knn_scores)
        best_k = k

print(f"KNN: mean acc - {best_score}, std - {best_score_std} at {best_k}")

100%|██████████| 95/95 [00:03<00:00, 25.66it/s]

KNN: mean acc - 0.8152941176470587, std - 0.017290515831410686 at 11


In [15]:
C_values = np.arange(5, 1000, 5)

best_score = 0
best_score_std = 0
best_C = 0

for C in tqdm(C_values):
    model = LogisticRegression(C=C)
    scores = run_cv(X, y, model)
    avg_score = np.mean(knn_scores)
    
    if avg_score > best_score:
        best_score = avg_score
        best_score_std = np.std(knn_scores)
        best_C = k

print(f"LogReg: mean acc - {best_score}, std - {best_score_std} at {best_C}")

100%|██████████| 199/199 [00:03<00:00, 55.26it/s]

LogReg: mean acc - 0.748235294117647, std - 0.03556756813607288 at 99


Получаем, что KNN лучше, поэтому для подсчета сабмита используем именно его

In [16]:
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X, y)

X_test = test_df[feature_cols]
X_test = scaler.fit_transform(X_test)
preds = knn.predict(X_test)

subm = pd.DataFrame()
subm["PassengerId"] = test_df["PassengerId"]
subm["Survived"] = preds.astype("int")
subm.to_csv("hw4/baseline_knn.csv", index=False)

Данный сабмит дает нам скор:

- public - 0.79781
- private - 0.79347